In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2023-08-01'
final_date = '2023-08-31'
query_error_year = 2023
query_error_month = 8

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\08. AGOSTO\DATA\ERROR AGOSTO 2023.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
# query_error.loc[((query_error['day'] == 30 )
#                 & (query_error['month'] == 7)
#                 & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
# query_error.loc[(query_error['day'] == 31 )
#             & (query_error['month'] == 7 )
#             & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_18776\960431041.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2023\08. AGOSTO\DATA\ERROR AGOSTO 2023.csv',


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(144663, 17)

In [5]:
query_error['mes_cierre'].unique()

array([nan, 'Actual'], dtype=object)

In [6]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [7]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [8]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [9]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_18776\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
7435,7435,762656fe-f025-4045-83e8-cfb2490d8955,Ma de Gracia Roque Díaz de León,D5D9VDW99,2020-02-10,9.58,ERROR,DEVOLUCION(03): Cuenta cancelada 030222...,2023-08-01,8,2023,NEW,ERROR_USUAL,762656fe-f025-4045-83e8-cfb2490d8955,2023-08-01,1,Actual
133016,133016,b799fede-5550-49c8-b8db-68c0b863ce11,NaN,A7FBGC9KT,2023-08-01,156.42,ERROR,/21/CB Return Account type is not applicable (...,2023-08-01,8,2023,NEW,ERROR_USUAL,b799fede-5550-49c8-b8db-68c0b863ce11,2023-08-01,1,Actual
133015,133015,0cebfd0b-c31f-4628-8dc4-4fbc35e80d0d,NaN,AQ9N9J9J7,2023-08-01,51.95,ERROR,/21/CB Return Account type is not applicable (...,2023-08-01,8,2023,NEW,ERROR_USUAL,0cebfd0b-c31f-4628-8dc4-4fbc35e80d0d,2023-08-01,1,Actual
133014,133014,2831bb2a-d361-45d6-b89b-7a454c55120b,NaN,AZRMZ4596,2023-08-01,498.29,ERROR,/21/CB Return Account type is not applicable (...,2023-08-01,8,2023,NEW,ERROR_USUAL,2831bb2a-d361-45d6-b89b-7a454c55120b,2023-08-01,1,Actual
133013,133013,694ec0fc-899b-4220-a020-2e48f01f870b,NaN,AY3JX5V36,2023-08-01,295.13,ERROR,/21/CB Return Account type is not applicable (...,2023-08-01,8,2023,NEW,ERROR_USUAL,694ec0fc-899b-4220-a020-2e48f01f870b,2023-08-01,1,Actual


In [10]:
query_error_current_month.shape

(13312, 17)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# AGRUPADO POR ESQUEMA

In [14]:
group_esquema = pd.pivot_table(query_error_current_month,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only      amount                                 \
esquema_query                      ERROR_USUAL            SIN_CLABE             
status_cubeta                            ERROR        NEW     ERROR       NEW   
0                       2023-08-01    3,525.34 580,561.27      0.00 96,776.14   
1                       2023-08-02        0.00 271,543.13      0.00 50,279.94   
2                       2023-08-03    3,589.94  21,429.59      0.00 53,032.97   
3                       2023-08-04   50,823.56  96,409.77      9.58 67,166.47   
4                       2023-08-05       23.00  80,106.86      0.00 63,723.21   

                          
esquema_query        All  
status_cubeta             
0             680,862.75  
1             321,823.07  
2              78,052.50  
3             214,409.38  
4             143,853.07

In [15]:
group_esquema.shape

(32, 6)

In [16]:
writer = pd.ExcelWriter('FASE VI. DATA POLIZA ERROR-SIN CLABE AGOSTO 2023.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
query_error_current_month.to_excel(writer, sheet_name='DETALLE',index=False)
writer.close()